In [1]:
import os, sys, shutil, time, pickle, glob
from pathlib import Path

In [2]:
root = "/home/gologors/data/"

os.chdir(root + "anno/")

print(os.getcwd())

children_folders = [fn for fn in os.listdir() if os.path.isdir(fn)]
# print(*[f"{i}: {fn}" for i,fn in enumerate(children_folders)], sep="\n")

/home/gologors/data/anno


In [15]:
segm_fns = list(glob.iglob(root + "anno/" + "/**/*.obj", recursive = True))

In [20]:
def get_nii_fn(ofn):
    parent_folder = str(Path(ofn).parent)
    sisters = list(os.listdir(parent_folder))
    niis = [fn for fn in sisters if fn.endswith(".nii") or fn.endswith(".nii.gz")]
    if len(niis) == 0:
        print("Error: ", ofn)
    return parent_folder + "/" + niis[0]

nii_fns = [get_nii_fn(ofn) for ofn in segm_fns]

In [22]:
print("N = ", len(segm_fns), " obj", len(nii_fns), " ims")

all_filenames = list(zip(nii_fns, segm_fns))

# Save
with open(root + 'pitmri/' + 'all_filenames.pkl', 'wb') as f:
    pickle.dump(all_filenames, f)

N =  50  obj 50  ims


In [23]:
# read back

with open(root + 'pitmri/' + 'all_filenames.pkl', 'rb') as f: 
    all_filenames = pickle.load(f)

In [24]:
# Split into training/valid and testing 
# adapted from https://github.com/Project-MONAI/tutorials/blob/main/modules/autoencoder_mednist.ipynb

test_frac = 0.2
valid_frac = 0.2

num_test  = int(len(all_filenames) * test_frac)
num_valid = int(len(all_filenames) * valid_frac)
num_train = len(all_filenames) - num_test - num_valid

train_datadict = [{"im": nii, "lbl":obj} for nii,obj in all_filenames[:num_train]]
valid_datadict = [{"im": nii, "lbl":obj} for nii,obj in all_filenames[num_train:num_train+num_valid]]
test_datadict = [{"im": nii, "lbl": obj} for nii,obj in all_filenames[-num_test:]]

print(f"total number of images: {len(all_filenames)}")
print(f"number of images for training: {len(train_datadict)}")
print(f"number of images for val: {len(valid_datadict)}")
print(f"number of images for testing: {len(test_datadict)}")

total number of images: 50
number of images for training: 30
number of images for val: 10
number of images for testing: 10
